In [1]:
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append('/net/mulan/home/huiwann/spatial/SVC/')
import tifffile
from model.train import *
from model.utils import *
import torch 
from torch.utils.data import DataLoader
import numpy as np
import random

In [2]:
data_path = '/net/mulan/home/huiwann/spatial/SVC/' 
dataset = 'data/merfish_U2OS' 
ckpt_folder = f'{data_path}checkpoints/'
device = 'cuda:1'

seed = 2025
torch.manual_seed(seed) 
random.seed(seed) 
np.random.seed(seed)

In [3]:
train_merfish_U2OS = np.load(f"{data_path}{dataset}/train_merfish_U2OS.npz") 
train_image = train_merfish_U2OS["data_ori"]
train_cell_morphology = train_merfish_U2OS["cell_morphology"]
train_nuclear_morphology = train_merfish_U2OS["nuclear_morphology"]
train_data_location = train_merfish_U2OS["location"]

train_dataset = SVC_Dataset(
    data_ori = train_image,
    cell_morphology_vec = train_cell_morphology,
    nuclear_morphology_vec = train_nuclear_morphology,
    location = train_data_location
)
print("number of training cells:", len(train_dataset),', number of genes:', train_image.shape[1])
cell_median_train = np.median(train_image.sum((1,2,3)))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4) 

number of training cells: 900 , number of genes: 119


In [4]:
read_dir =f'{data_path}{dataset}/gene2vec_weight_merfish_U2OS.npy'
gene2vec_weight = torch.from_numpy(np.load(read_dir)).float() ##n_gene * 200
print("shape of gene2vec_weight", gene2vec_weight.shape)
model = SVC(
    gene2vec_weight = gene2vec_weight,
    use_cell_identity = False,
).to(device)


shape of gene2vec_weight torch.Size([119, 200])


In [5]:
epoch_losses = train_SVC(
    model = model,
    train_loader = train_loader,
    cell_median_train = cell_median_train,
    device = device,
    num_epochs = 400,
    ckpt_dir = ckpt_folder,
    ckpt_name = "SVC_merfish_U2OS",
    use_cell_identity = False,
    early_stop_patience = 50
)


Training:   0%|          | 0/400 [00:00<?, ?epoch/s]

Early stopping at epoch 207 with best loss 90.9365
Saving model checkpoint to /net/mulan/home/huiwann/spatial/SVC/checkpoints/SVC_merfish_U2OS.pth
Finished training
